# Real Time Tweet Analysis & Dashboarding Tool (1/2)
## Project Motivation
Originally, my idea for this project was to stream tweets about the Washington Redskins, analyze their polarity scores, and visualize the overall negative/positive sentiment towards the team in real time.  As I learned more about Python's Tweepy library, I realized that it would be easy to analyze any word a user wanted.  Although I could simply set 'Redskins' as the tracked word, I thought it would be a better idea to allow the user to specify the input.  Once specified, Tweepy will start to stream tweets containing the tracked word.  The streamed tweets will then be cleaned and various aspects will be displayed on a dashboard that can be updated to get real time data.  

## How to Use the Tool

First, go to https://developer.twitter.com/en/account/get-started to access your Twitter application's keys.  Before running this notebook, these keys need to be pasted into the cell below titled "Connection to Twitter API".  If you do not already have a Twitter Developer account or app, feel free to set up an account.  Once you have an account and an app created, you can access the keys you need to run this notebook.

This project has two jupyter notebooks that work together.  If you run all cells in this notebook, 'DATS 6103 - Project 3 Streaming', you will be prompted to enter a tracked word to analyze.  Once a tracked word is entered, this notebook will stream tweets containing the tracked word, and write them to a .csv file called 'OutputStreaming.csv'.  After choosing your word, you can scroll to the bottom to see the tweets being streamed.

Running all cells in the second notebook, 'DATS 6103 - Project 3 Analysis.ipynb' reads the .csv file into a dataframe and performs the analysis.  The end of this second file gives a dashboard showing real time information about Tweets containing the tracked word.  One can keep running all cells in the 'DATS 6103 - Project 3 Analysis.ipynb' notebook to update the dashboard as new Tweets are constantly being streamed from this notebook.

## Importing Libraries

In [68]:
import tweepy

import pandas as pd

import re
import itertools
import collections

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from textblob import TextBlob

import sys
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonathangiguere/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Enter the Word You Would like to Dashboard

In [56]:
print('Enter the word you would like to track on Twitter.')
tracked_word = input()

Enter the word you would like to track on Twitter.
impeachment


## Connection to Twitter API

In [57]:
#connection to twitter
consumer_key='xxxxxxxxxxxxxxxxxxxxxxxxxx'
consumer_secret='xxxxxxxxxxxxxxxxxxxxxxxxxx'
access_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'
access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'

#use variables to access twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

## Functions for Text Preprocessing
The code in this section consists of a few functions I wrote to clean the tweets of URLs, mentions, emojis, hashtags, stopwords, etc.

In [58]:
#Function to remove URLs and mentions
def clean_status(status): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", status).split()) 

In [59]:
#Function to remove emojis
def clean_emojis(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [60]:
#Function to remove hashtags and any other remaining special characters
def remove_hashtags(text):
    newstr = text.replace("#", "")
    newstr = newstr.replace("_", "")
    newstr = newstr.replace(":", "")
    newstr = newstr.replace(")", "")
    newstr = newstr.replace("(", "")
    newstr = newstr.replace('"', "")
    newstr = newstr.replace("RT", "")
    newstr = newstr.replace("!", "")
    newstr = newstr.replace("@", "")
    newstr = newstr.replace("%", "")
    newstr = newstr.replace("-", "")
    return newstr

In [61]:
#Cache stop words
stop_words = set(stopwords.words('english'))

#Remove stop words from each tweet
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

## Creating StreamListener
The code below creates the StreamListener object.  

#### on_status
- The 'on_status' method actually gets the tweets and does all of the text pre-processing by calling the functions I have written.  It also calculates polarity scores for each tweet as they come in.  
- The method attempts to get longitude and latitude for each incoming tweet but not many people have location for every tweet turned on.  
- The method prints the unprocessed tweet text as ouput and also writes each tweets values (text, username, polarity score, location, etc.) to a file called 'OutputStreaming.csv' row by row.

#### on_error
- This method gets error codes and returns False by default.  Returning False would disconnect the stream so I made it return True always.  Returning True reconnects the stream with back-off.  

In [62]:
#create the Stream Listener object
class CustomStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        #Assesses whether each tweet is a retweet or not to be stored in a column
        if status.text.startswith("RT @"):
            retweet = True
        else:
            retweet = False
        
        #Clean tweet of URLs and @usernames
        status_text = clean_status(status.text)
        
        #Clean tweets of emojis
        status_text = clean_emojis(status_text)
        
        #Remove hashtags and any other remaining non-text characters
        status_text = remove_hashtags(status_text)
        
        polarity_sw = TextBlob(status_text).sentiment.polarity
        
        #Remove stop words from tweet
        status_text = remove_stopwords(status_text)
        
        #Get sentiment score for the tweet to be stored in a column
        polarity_nsw = TextBlob(status_text).sentiment.polarity
        
        #Gets longitude and latitude of tweet if it exists
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
        
        #Print out raw text of tweets as they stream
        print(status.text)
        
        # Writing tweets to a .csv
        with open('OutputStreaming.csv', 'a') as f:
            writer = csv.writer(f)
            #All of the data to be written in each row
            writer.writerow([status.author.screen_name, status.created_at, status_text, 
                             retweet, polarity_sw, polarity_nsw, status.user.location, 
                             status.user.followers_count, longitude, latitude])


    def on_error(self, status_code):
        #Returning True here keeps the stream going
        return True 

## Creating OutputStreaming.csv and Handing the Tracked Word to StreamListener
This block of code creates the .csv file to store the tweets and assigns appropriate column names.  We use our StreamListener object created above and give it parameters to pull tweets in English based on the tracked word chosen by the user.

In [63]:
with open('OutputStreaming.csv', 'w') as f:
    writer = csv.writer(f)
    #Give names for first row of .csv
    writer.writerow(['Author', 'Date', 'Text', 'Retweet?', 'Polarity_SW', 'Polarity_NSW',
                    'User Location', 'User_Follower_Count', 'Longitude',
                    'Latitude'])

streamingAPI = tweepy.streaming.Stream(auth, CustomStreamListener())

#Specify English tweets and choose any word/brand to track
streamingAPI.filter(languages=["en"], track=[tracked_word])

@RepJayapal I will take your vote for impeachment as a personal attack on my right to choose my president. No matte… https://t.co/GG6k7WPo0O
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @joeblackzw: Knowledge bombs, this thread. Killer last line, too. https://t.co/Q5W76Fj3Su
RT @Jim_Jordan: They did it again.

This afternoon at 4:45, Democrats said Republicans must submit amendments to their ridiculous impeachme…
RT @RepMattGaetz: There are a lot of moderate Dems who recognize what a political loser impeachment is.

They're encouraging their leadersh…
@AndrewDesiderio @NatashaBertrand Since articles of impeachment can be amended for new charges or other pertinent m… https://t.co/wddRxjfgiP
RT @joshchafetz: oh god please someone ask him what he thinks he 

RT @RNCResearch: Rep. Lesko slams impeachment as a “corrupt, rigged, railroad job”
https://t.co/0rMt58fMat https://t.co/SW28CMASPX
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
RT @RepMattGaetz: There are a lot of moderate Dems who recognize what a political loser impeachment is.

They're encouraging their leadersh…
RT @no_silenced: @realDonaldTrump Pennsylvania is yours Mr President....How many Democrats In The House Are from Pennsylvania and are gonna…
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
Rep. Louie Gohmert Says Name of Alleged Ukraine Whistleblower at House Judiciary Committee Impeachment Meeting… https://t.co/VJIZ5HJ1NQ
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
RT @CNN: Senate Republicans are starting

RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
RT @dbongino: FLOOD THE PHONE LINES! Be sure to call your congressional representative and express your outrage with this sham impeachment…
RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @drefanzor: Adam Schiff training for the impeachment sham. https://t.co/nDNKZyxsAx
RT @GOPChairwoman: Pelosi is putting every Democrat politician on the spot for supporting impeachment.
 
@MarshaBlackburn is right: It’s go…
RT @AdamParkhomenko: This dumb motherfucker got elected to Congress. If you’re thinking about running for office but doubt whether you can,…
RT @RepMattGaetz: There are a lot of moderate Dems who recognize what a political loser impeachment is.

They're encouraging their leadersh…
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN i

RT @RepMikeJohnson: The same Democrats who once warned against a single party #impeachment, are here today leading the partisan witch-hunt…
RT @fred_guttenberg: (1,4) I HAVE A QUESTION FOR MY LEGAL FRIENDS.  As the facts keep coming out, I have a question for the Jurors (Senator…
RT @AndrewDesiderio: SCOOP: Pence’s counsel tells Schiff late tonight it “serves no purpose” to declassify Jennifer Williams’ supplemental…
RT @KevinMKruse: I'm sorry, did @RepRatcliffe seriously just say "abuse of power" and "obstruction of Congress" had never been sustained in…
🤦🏻‍♂️ dems more focused on this impeachment, rather than fixing the country. When he’s re-elected, what are they go… https://t.co/ecWyt1gXSx
RT @abigailmarone: Rep. Ted Deutch's reason for impeaching @realDonaldTrump is that 𝙝𝙞𝙨 𝙠𝙞𝙙𝙨 𝙞𝙣 𝙝𝙞𝙨 𝙛𝙖𝙢𝙞𝙡𝙮 𝙜𝙧𝙤𝙪𝙥 𝙩𝙚𝙭𝙩 𝙛𝙚𝙚𝙡 𝙨𝙖𝙙.

THIS is ho…
Trump impeachment markup gets heated as top Dem Nadler says 'we cannot rely on an election' | Fox News https://t.co/dqA1FSwbSU
RT @realDonaldTrump: “New Pol

RT @RepMattGaetz: Impeachment has become reflexive for Dems. It’s what they’ve wanted all along.

Impeachment is their passion, their drug,…
RT @RepMattGaetz: This is the fastest, weakest, thinnest, MOST PARTISAN impeachment in American presidential history. #impeachmentDebate
RT @RepMattGaetz: There are a lot of moderate Dems who recognize what a political loser impeachment is.

They're encouraging their leadersh…
RT @mel_faith1: Let it out big guy, let it out!

Impeachment hoaxing is hard work. So is sitting and staring awkwardly without blinking.…
RT @GOPLeader: While Dems distract the country with impeachment, here's what we've learned:
→ A law enforcement agency spied on President T…
RT @OhNoSheTwitnt: If these impeachment hearings have taught us anything it’s that Republican men are too emotional and hysterical to lead.
@RepMattGaetz @realDonaldTrump Sometimes I ask if America can ever overcome this impeachment.

It has been destroye… https://t.co/gNKjz88PHt
RT @RepMattGaetz: Imp

...

RT @GovMikeHuckabee: Memo to my friend @LindseyGrahamSC remember what Margaret Thatcher said to Pres Bush: "Don't go wobbly on us, George!"…
@MooreHomes1 @washingtonpost Fox News says 55% suppprt impeachment.  Impeach AND Remove support up 9 points since J… https://t.co/VtNnaj1PhL
RT @TheDemCoalition: Reminder: The president is a thrice married philander who cheated on his third wife with a porn star while she was nur…
Important and timely thread: 👇👇👇👇
RT @neeratanden: I would just like to remind everyone that every time something terrible happens to Trump, his folks argue it makes him big…
RT @RepMattGaetz: Impeachment has become reflexive for Dems. It’s what they’ve wanted all along.

Impeachment is their passion, their drug,…
#RussianizedRepublicans -  the corrupt protecting the corrupt. #ImpeachAndConvictTrump
RT @charliekirk11: BREAKING:

Some House Democrats have announced they will continue to pursue impeachment against President Trump even AFT…
RT @Jim_Jordan: They did it again

RT @RepMattGaetz: There are a lot of moderate Dems who recognize what a political loser impeachment is.

They're encouraging their leadersh…
Impeachment hot take: https://t.co/5AnAgEEDvq
RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @RepAndyBiggsAZ: .@RepMattGaetz shows how partisan the Democrats' lead investigator, Mr. Goldman, is a partisan hack.

There is NOTHING…
RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @RepMattGaetz: Impeachment has become reflexive for Dems. It’s what they’ve wanted all along.

Impeachment is their passion, their drug,…
RT @TeaPainUSA: FUN FACT: Impeachment is pollin’ 8-10 points higher than Trump’s approval ratin’. https://t.co/jhGTOd05Sg
RT @dbongino: New Poll: Majority of Americans Oppose Impeachment  https://t.co/jzP3SCKNo2
RT @RepAndyBiggsAZ: .@RepMattGaetz sho

RT @dbongino: FLOOD THE PHONE LINES! Be sure to call your congressional representative and express your outrage with this sham impeachment…
The Republicans continued obstruction will be addressed before this is all over.
RT @GOPChairwoman: Pelosi lied to the country by promising to get things done.

Americans wanted from Democrats:
*lower prescription drug c…
RT @dbongino: FLOOD THE PHONE LINES! Be sure to call your congressional representative and express your outrage with this sham impeachment…
RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @MyBigRedTruck: GOP Rep. Louie Gohmert publicly identifies person purported to be whistleblower
https://t.co/xQ8F8tn6PN

Republican scum…
RT @sarahkendzior: They've both been accused of raping teenagers procured for them by pedophile trafficker Jeffrey Epstein https://t.co/I7S…
RT @RepBrendanBoyle: I applaud my hometown newspaper @PhillyInquirer for of

RT @realDonaldTrump: “New Polls Say Most Americans Oppose Impeachment.” @foxandfriends  I did nothing wrong. This will be the first Impeach…
RT @TrumpWarRoom: Democrat Hank Johnson falsely claimed tonight that he didn’t support impeachment until just recently.

In fact, he was pu…
RT @Jamierodr14: WOKE!

Democrat in Alabama 

"After watching the Brett Kavanaugh hearings, that was the most disgusting thing I've seen. N…
But it's still impeachment.
RT @RepDougCollins: .@RepMikeJohnson: It’s rather shocking that Democrats built their impeachment articles on mere hearsay, speculation, an…
RT @charliekirk11: Facts:

Democrats put off passing the USMCA for over a year

They haven’t done anything to fix healthcare

They refuse t…
RT @brat2381: Do these nutjobs really think this country is going to go through another Civil War for a draft dodging, veteran charity defr…
RT @DevinCow: Did anybody tell OJ about Dershowitz possibly joining this legal team? OJ blocked me when I reminded him that he

KeyboardInterrupt: 